<a href="https://colab.research.google.com/github/abdinegara783/FindIT2023/blob/Experiment/FindIT_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import xgboost as xgb

from google.colab import drive
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [40]:
# Mount Google Drive to access dataset
drive.mount('/content/drive', force_remount=True)

# Set up directory path
file_path = '/content/drive/MyDrive/df_TrainingFeatures.csv'
target_path = '/content/drive/MyDrive/training_set_labels.csv'

Mounted at /content/drive


In [41]:
# Read CSV file
data = pd.read_csv(file_path)

# Print the contents of the CSV file
data.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty,employment_status_Employed,employment_status_Not in Labor Force,employment_status_Unemployed,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
0,0,1,0,0,0,0,0,0,1,1,...,1,0,0,1,0,1,0,0,0,1
1,1,3,2,0,1,0,1,0,1,1,...,1,0,0,1,1,0,0,1,0,0
2,2,1,1,0,1,0,0,0,0,0,...,1,1,0,0,1,0,0,1,0,0
3,3,1,1,0,1,0,1,1,0,0,...,1,0,0,1,0,1,0,0,1,0
4,4,2,1,0,1,0,1,1,0,1,...,1,1,0,0,1,0,0,1,0,0


In [42]:
# Read CSV file
target = pd.read_csv(target_path)

# Print the contents of the CSV file
target.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [43]:
# merge the dataframes using respondent_id as primary key
merged_df = pd.merge(data, target, on='respondent_id')

# print the merged dataframe
merged_df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,"income_poverty_> $75,000",income_poverty_Below Poverty,employment_status_Employed,employment_status_Not in Labor Force,employment_status_Unemployed,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA,h1n1_vaccine,seasonal_vaccine
0,0,1,0,0,0,0,0,0,1,1,...,0,1,0,1,0,0,0,1,0,0
1,1,3,2,0,1,0,1,0,1,1,...,0,1,1,0,0,1,0,0,0,1
2,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,3,1,1,0,1,0,1,1,0,0,...,0,1,0,1,0,0,1,0,0,1
4,4,2,1,0,1,0,1,1,0,1,...,0,0,1,0,0,1,0,0,0,0


In [44]:
merged_df.to_csv('data_and_target.csv', index=False)

In [45]:
# copy dataframe df menjadi df1 dan df2
df1 = merged_df.copy()
df2 = merged_df.copy()

# **Code for Target 1**

In [46]:
# memisahkan feature dan label
X = df1.iloc[:, :-1].astype('int')
y = df1.iloc[:, -1].astype('int')

# mengatur ulang nama fitur
X.columns = ['column_'+str(i) for i in range(X.shape[1])]

# membagi dataset menjadi train set dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# membangun model XGBoost
model = xgb.XGBClassifier()

# menentukan hyperparameter yang akan dituning
hyperparameter = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200],
}

# melakukan grid search pada model dengan hyperparameter yang telah ditentukan
grid_search = GridSearchCV(estimator=model, param_grid=hyperparameter, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# mencetak hasil grid search
print('Best hyperparameter:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

Best hyperparameter: {'learning_rate': 0.1}
Best score: 0.7981277562847885


In [47]:
# membuat objek model baru dengan parameter terbaik
best_model = xgb.XGBClassifier(**grid_search.best_params_)

# melatih ulang model pada data latih
best_model.fit(X_train, y_train)

# melakukan prediksi pada data uji
X_test.columns = [col.replace('[', '_').replace(']', '') for col in X_test.columns]
y_pred = best_model.predict(X_test)

# menghitung akurasi model pada data uji
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8101834518906776


# **Code for Target 2**

In [48]:
# memisahkan feature dan label
X = df2.iloc[:, :-1].astype('int')
y = df2.iloc[:, -1].astype('int')

# mengatur ulang nama fitur
X.columns = ['column_'+str(i) for i in range(X.shape[1])]

# membagi dataset menjadi train set dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# membangun model XGBoost
model = xgb.XGBClassifier()

# menentukan hyperparameter yang akan dituning
hyperparameter = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200],
}

# melakukan grid search pada model dengan hyperparameter yang telah ditentukan
grid_search = GridSearchCV(estimator=model, param_grid=hyperparameter, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# mencetak hasil grid search
print('Best hyperparameter:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

Best hyperparameter: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best score: 0.8035105010022985


In [49]:
# membuat objek model baru dengan parameter terbaik
best_model = xgb.XGBClassifier(**grid_search.best_params_)

# melatih ulang model pada data latih
best_model.fit(X_train, y_train)

# melakukan prediksi pada data uji
X_test.columns = [col.replace('[', '_').replace(']', '') for col in X_test.columns]
y_pred = best_model.predict(X_test)

# menghitung akurasi model pada data uji
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8111194309247473
